In [1]:
using DataFrames
using JuMP, Cbc, Xpress

┌ Info: Xpress: Found license file xpauth.xpr
└ @ Xpress C:\Users\lucasresck\.julia\packages\Xpress\Jdi4R\src\xprs_userlic.jl:43
┌ Info: Xpress: User license detected.
└ @ Xpress C:\Users\lucasresck\.julia\packages\Xpress\Jdi4R\src\xprs_userlic.jl:98
┌ Info: Xpress-MP licensed by Fair Isaac Corporation to PSR for PSR applications
└ @ Xpress C:\Users\lucasresck\.julia\packages\Xpress\Jdi4R\src\xprs_userlic.jl:99


In [32]:
include("../../Functions\\dados_e_plot.jl");

In [2]:
#Valores padrão
G = [10, 5, 20, 18, 23, 32, 7, 12, 10, 20, 17, 32, 12, 13, 17]
d = [238, 220, 187, 175, 135, 127, 144, 165, 193, 205, 213, 233]
R = [33, 46, 32, 27, 10, 8, 20, 21, 37, 41, 27, 27]
θ = 0.2
Ns = 10
K = Int.(ones(length(G)))
cg = Int.(zeros(length(G)))

d_array = [[Int(round(d * 0.7)) for d in d]]
for scenario in 2:Ns
    push!(d_array, [Int(round(d * (0.4 + 0.6 * rand()))) for d in d])
end
dsR = hcat(d_array...);

# 1.8 Probabilidade de falha

## 1.8.1 Soma de variáveis aleatórias

Dadas duas variáveis independentes $X$ e $Y$ e suas probabilidades $f_X(x)$ e $f_Y(y)$, respectivamente, como calcular $f_Z(z)$ onde $Z = X + Y$?

Calculamos a probabilidade conjunta $f(x, y) = f_X(x)f_Y(y)$.

Seguimos com $F_Z(z) = P(Z \leq z) = P(X + Y \leq z) = \int\int_{R}f(x, y)dA$, com $R$ definida pela região $X + Y \leq z$. Temos, portanto, a probabilidade acumulada de $X + Y \leq z$.

Temos também $F_Z(z) = \int_{-\infty}^z f_Z(t)dt$. Deriva-se $F_Z(z)$ encontrado e tem-se $f_Z(z)$.

## 1.8.2 Implementação para $2$ usinas

Uma implementação possível para duas usinas é calcular a priori as probabilidades para todos os casos de manutenção. 

$f_i$ é uma variável auxiliar que indica que a usina $i$ está em manutenção ($0$ corresponde à manutenção); $P_c$ é a probabilidade da demanda ser suprida em um dado cenário de manutenção; $P_j$ é a probabilidade da demanda ser suprida no cenário de manutenção $j$; $\gamma_j$ é uma variável auxiliar da convolução. Segue:

$$min\ \ G_{disp}$$

$$s.a.\ \ G_{disp} = \sum G_i f_i$$
$$P_c \geq \epsilon$$
$$P_c = \sum_{j = 0} P_j \gamma_j$$
$$\sum_{j = 0} \gamma_j = 1$$

$$\gamma_j \geq 0, \forall j$$
$$\gamma_0 \geq 1 - f_1 - f_2$$
$$\gamma_1 \geq f_1 - f_2$$
$$\gamma_2 \geq f_2 - f_1$$
$$\gamma_3 \geq f_1 + f_2 - 1$$

Sendo assim, como $\sum_{j = 0} \gamma_j = 1$, apenas um $\gamma_j$ será $1$, de modo que $P_c = \sum_{i = 0} P_i \gamma_i$ será igual a apenas um $P_j \gamma_j$. Ou seja, o solver escolhe aquele caso (ou um daqueles casos) em que $P_j \geq \epsilon$.

Sejam duas usinas com capacidades disponíveis $G_1 = 30$ e $G_2 = 40$ e uma demanda $D = 35$.

In [3]:
G8_2 = [30, 40]
D8_2 = 35
p8_2 = [0.5, 0.5]
ϵ8_2 = 0.3 #Probabilidade de falha de cada gerador
nplants8_2 = length(G8_2)
nmonths8_2 = size(D8_2);

In [4]:
tabela8_2 = DataFrames.DataFrame(usinas = [0, 1, 2, [1,2]], probabilidade = [0, 0, 0.5, missing])

,usinas,probabilidade
,Any,Float64⍰
1,0,0.0
2,1,0.0
3,2,0.5
4,"[1, 2]",missing


In [5]:
F_Z8_2 = 0
for X in [0, 1]
    for Y in [0, 1]
        Z = G8_2[1] * X + G8_2[2] * Y
        if Z < D8_2
            F_Z8_2 += (X + (-1)^X * p8_2[1]) * (Y + (-1)^Y * p8_2[2])
        end
    end
end
tabela8_2.probabilidade[4] = 1 - F_Z8_2
tabela8_2

,usinas,probabilidade
,Any,Float64⍰
1,0,0.0
2,1,0.0
3,2,0.5
4,"[1, 2]",0.5


In [37]:
m8_2 = Model(solver = Xpress.XpressSolver())

@variable(m8_2, f[1:nplants8_2], Bin)
@variable(m8_2, γ[1:nplants8_2^2], Bin)
@variable(m8_2, G_disp)
@variable(m8_2, Pc)

@constraint(m8_2, G_disp == sum(G8_2 .* f))
@constraint(m8_2, Pc >= ϵ8_2)
@constraint(m8_2, Pc == sum(tabela8_2.probabilidade .* γ))
@constraint(m8_2, sum(γ) == 1)

@constraint(m8_2, γ[1] >= 1 - f[1] - f[2])
@constraint(m8_2, γ[2] >= f[1] - f[2])
@constraint(m8_2, γ[3] >= f[2] - f[1])
@constraint(m8_2, γ[4] >= f[1] + f[2] - 1)

@objective(m8_2, Min, G_disp);

In [38]:
@time solve(m8_2)
@show getvalue(f)
@show getvalue(γ)
@show getvalue(G_disp)
@show getvalue(Pc);

  0.003415 seconds (115 allocations: 12.555 KiB)
getvalue(f) = [-0.0, 1.0]
getvalue(γ) = [-0.0, -0.0, 1.0, -0.0]
getvalue(G_disp) = 40.0
getvalue(Pc) = 0.5


## 1.8.3 Implementação para $n$ usinas

Calcula-se a priori as probabilidades para todos os casos de manutenção. 

$f_i$ é uma variável auxiliar que indica que a usina $i$ está em manutenção ($0$ corresponde à manutenção); $P_c$ é a probabilidade da demanda ser suprida em um dado cenário de manutenção; $P_j$ é a probabilidade da demanda ser suprida no cenário de manutenção $j$; $\gamma_j$ é uma variável auxiliar para o cálculo. Segue:

$$min\ \ G_{disp}$$

$$s.a.\ \ G_{disp} = \sum G_i f_i$$
$$P_c \geq \epsilon$$
$$P_c = \sum_{j = 0} P_j \gamma_j$$
$$\sum_{j = 0} \gamma_j = 1$$

$$\gamma_j \geq 0, \forall j$$
$$\gamma_j \geq \sum_{k \in K_j} f_k - \sum_{m \in M_j} f_m - |K_j| + 1, \ \ \forall j$$

sendo $K_j$ o conjunto de usinas disponíveis no cenário de manutenção $j$, $M_j$ o conjunto de usinas em manutenção no cenário $j$ e $|K_j|$ o número de usinas disponíveis no cenário $j$.

In [8]:
G8_3 = G
D8_3 = 180
ϵ8_3 = 0.5
p8_3 = rand(0:2, 15) ./ 10
nplants8_3 = length(G8_3);

In [9]:
function prob(G, p, D)
    F_Z = 0
    nplants = length(G)
    for config in 0:(2^nplants - 1) #Para cada configuração de falha
        config_bin = Base.bin(Unsigned(config), nplants, false) #Converte para binário
        X = [parse(Int, config_bin[end - plant + 1]) for plant in 1:nplants] #Converte para array
        Z = sum(G .* X)            
        if Z < D
            F_Z += prod(X .+ (-1).^X .* p)
        end
    end
    return 1 - F_Z
end;

In [10]:
function tabela(G, p, D)
    nplants = length(G)
    tabela = zeros(2^nplants, 2)
    tabela = convert(Array{Any}, tabela)
    
    for config in 0:(2^nplants - 1)
        config_bin = Base.bin(Unsigned(config), nplants, false)
        X = [parse(Int, config_bin[end - plant + 1]) for plant in 1:nplants] #Converte para array
        Z = sum(G .* X)
        if Z >= D
            tabela[config + 1, 1] = config_bin
            G_aux = []
            p_aux = []
            for plant in 1:nplants
                if config_bin[end + 1 - plant] == '1'
                    push!(G_aux, G[plant])
                    push!(p_aux, p[plant])
                end
            end
            if length(G_aux) == 0 #Se o número de usinas é zero
                probability = 0.0
            else
                probability = prob(G_aux, p_aux, D)
            end
            tabela[config + 1, 2] = probability
        end
    end
    return tabela
end;     

In [11]:
@time tabela8_3 = tabela(G8_3, p8_3, D8_3);

 16.748463 seconds (239.69 M allocations: 7.109 GiB, 8.39% gc time)


In [12]:
m8_3 = Model(solver = Xpress.XpressSolver(OUTPUTLOG = 1))

@variable(m8_3, G_disp)
@variable(m8_3, f[1:nplants8_3], Bin)
@variable(m8_3, Pc)
@variable(m8_3, γ[1:2^nplants8_3], Bin)

@constraint(m8_3, G_disp == sum(G8_3 .* f))
@constraint(m8_3, Pc >= ϵ8_3)
@constraint(m8_3, Pc == sum(tabela8_3[:, 2] .* γ))
@constraint(m8_3, sum(γ) == 1)

for j in 0:(2^nplants8_3 - 1)
    j_bin = Base.bin(Unsigned(j), nplants8_3, false)
    j_array = [parse(Int, j_bin[end + 1 - plant]) for plant in 1:nplants8_3]
    @constraint(m8_3, γ[j + 1] >= sum((-1)^(1 + j_array[plant]) * f[plant] for plant in 1:nplants8_3) + 1 - count_ones(j))
end

@objective(m8_3, Min, G_disp);

In [13]:
JuMP.build(m8_3)
Xpress.setlogfile(m8_3.internalModel.inner,"D:/m8_3.log")

In [14]:
@time solve(m8_3)
@show getvalue(f)
#@show getvalue(γ)
@show getvalue(G_disp)
@show getvalue(Pc);

139.520741 seconds (133 allocations: 6.542 MiB)
getvalue(f) = [1.0, 1.0, 1.0, -0.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0]
getvalue(G_disp) = 210.0
getvalue(Pc) = 0.5015019847680001


## 1.8.4 Método de Monte Carlo

O cálculo da probabilidade utilizando o Método de Monte Carlo consiste em simular repetidamente o processo aleatório de falha das usinas, utilizando suas probabilidades de falha, e verificar se a demanda é suprida em cada caso. A probabilidade da demanda ser suprida é aproximadamente a razão entre o número de casos em que a demanda foi suprida na simulação e o número total de casos.

In [15]:
G8_4 = G8_2
p8_4 = p8_2
D8_4 = D8_2;

In [16]:
function prob_MC(G, p, D, n)
    nplants = length(G)
    count = 0
    for i in 1:n
        Z = 0
        for plant in 1:nplants
            if rand() > p[plant]
                Z += G[plant]
            end
        end
        if Z >= D
            count += 1
        end
    end
    return count / n
end;

In [17]:
function tabela_MC(G, p, D; n = 1000)
    nplants = length(G)
    tabela = zeros(2^nplants, 2)
    tabela = convert(Array{Any}, tabela)
    
    for config in 0:(2^nplants - 1)
        config_bin = Base.bin(Unsigned(config), nplants, false)
        tabela[config + 1, 1] = config_bin
        G_aux = []
        p_aux = []
        for plant in 1:nplants
            if config_bin[end + 1 - plant] == '1'
                push!(G_aux, G[plant])
                push!(p_aux, p[plant])
            end
        end
        if length(G_aux) == 0 #Se o número de usinas é zero
            probability = 0.0
        else
            probability = prob_MC(G_aux, p_aux, D, n)
        end
        tabela[config + 1, 2] = probability
    end
    return tabela
end;     

In [18]:
tabela8_4 = tabela_MC(G8_4, p8_4, D8_4, n = 10000)

4×2 Array{Any,2}:
 "00"  0.0   
 "01"  0.0   
 "10"  0.498 
 "11"  0.5097

In [19]:
G8_4_2 = G8_3
p8_4_2 = p8_3
D8_4_2 = D8_3;

In [20]:
@time tabela8_4_2 = tabela_MC(G8_4_2, p8_4_2, D8_4_2);

 13.192878 seconds (246.38 M allocations: 3.686 GiB, 7.80% gc time)


In [21]:
tabela8_3[(end - 10):end, :]

11×2 Array{Any,2}:
 "111111111110101"  0.687503
 "111111111110110"  0.64893 
 "111111111110111"  0.742136
 "111111111111000"  0.569009
 "111111111111001"  0.661128
 "111111111111010"  0.630421
 "111111111111011"  0.710215
 "111111111111100"  0.763878
 "111111111111101"  0.82695 
 "111111111111110"  0.800264
 "111111111111111"  0.860041

In [22]:
tabela8_4_2[(end - 10):end, :]

11×2 Array{Any,2}:
 "111111111110101"  0.68 
 "111111111110110"  0.642
 "111111111110111"  0.721
 "111111111111000"  0.56 
 "111111111111001"  0.639
 "111111111111010"  0.639
 "111111111111011"  0.724
 "111111111111100"  0.781
 "111111111111101"  0.846
 "111111111111110"  0.801
 "111111111111111"  0.852

## 1.8.5 Adição da obrigatoriedade de manutenção e vários estágios

Existem vários estágios, digamos, meses, e cada usina deve entrar em manutenção em um estágio. A probabilidade da demanda ser suprida deve ser maior do que ou igual a um parâmetro informado, para todo estágio. Maximiza-se a soma das probabilidades da demanda ser suprida nos estágios.

Calcula-se a priori as probabilidades para todos os casos de manutenção. 

$f_i^t$ é uma variável auxiliar que indica que a usina $i$ está em manutenção no estágio $t$ ($0$ corresponde à manutenção); $P_c^t$ é a probabilidade da demanda ser suprida em um dado cenário de manutenção no estágio $t$; $P_j^t$ é a probabilidade da demanda ser suprida no cenário de manutenção $j$, no estágio $t$; $\gamma_j^t$ é uma variável auxiliar para o cálculo. Segue:

$$max\ \ \sum_t P_c^t$$

$$s.a.\ \ P_c^t \geq \epsilon, \ \ \forall t$$
$$P_c^t = \sum_{j = 0} P_j \gamma_j^t, \ \ \forall t$$
$$\sum_{j = 0} \gamma_j^t = 1, \ \ \forall t$$
$$\sum_t f_i^t = |T| - 1, \ \ \forall i$$

$$\gamma_j^t \geq 0, \forall j, t$$
$$\gamma_j^t \geq \sum_{k \in K_j} f_k^t - \sum_{m \in M_j} f_m^t - |K_j| + 1, \ \ \forall j, t$$

sendo $T$ o conjunto de estágios, $|T|$ o número de estágios (exemplo, $12$ meses), $K_j$ o conjunto de usinas disponíveis no cenário de manutenção $j$, $M_j$ o conjunto de usinas em manutenção no cenário $j$ e $|K_j|$ o número de usinas disponíveis no cenário $j$.

In [23]:
G8_5 = G
d8_5 = [d[1] - 50; d[2:11]; d[12] - 50] #Esses meses têm demanda tão alta que, com os geradores, a probabilidade de suprir a demanda é muito baixa, para todos os cenários de manutenção
ϵ8_5 = 0.5
nplants8_5 = length(G8_5)
N_t8_5 = length(d8_5)
p8_5 = rand(0:2, nplants8_5) ./ 10;

In [24]:
function tabelas(G, p, d)
    N_t = length(d)
    tabelas = []
    for t in 1:N_t
        D = d[t]
        push!(tabelas, tabela(G, p, D))
    end
    return tabelas
end;

In [25]:
@time tabelas8_5 = tabelas(G8_5, p8_5, d8_5);

201.653012 seconds (3.00 G allocations: 89.791 GiB, 10.79% gc time)


In [26]:
[maximum(tabelas8_5[month][:, 2]) for month in 1:12]

12-element Array{Float64,1}:
 0.9338865868799999
 0.5724463104000019
 0.9437690879999998
 0.97997239808     
 0.99989395712     
 0.99998115392     
 0.99960437056     
 0.99315337472     
 0.91310846976     
 0.7983592243199997
 0.715451719679999 
 0.9574486220800001

In [27]:
tabelas8_5[1][end-10:end, 2]

11-element Array{Any,1}:
 0.7886499839999999
 0.7415930879999999
 0.8223768575999997
 0.6538060800000001
 0.806754816       
 0.7171752959999999
 0.8286188543999999
 0.8496138239999999
 0.9189098496      
 0.8900278271999998
 0.9338865868799999

In [40]:
m8_5 = Model(solver = Xpress.XpressSolver(OUTPUTLOG = 1, MIPRELSTOP = 0.04, MAXTIME = 3600))

@variable(m8_5, f[1:nplants8_5, 1:N_t8_5], Bin)
@variable(m8_5, Pc[1:N_t8_5])
@variable(m8_5, γ[1:2^nplants8_5, 1:N_t8_5], Bin)

for t in 1:N_t8_5
    @constraint(m8_5, Pc[t] >= ϵ8_5)
    @constraint(m8_5, Pc[t] == sum(tabelas8_5[t][:, 2] .* γ[1:2^nplants8_5, t]))
    @constraint(m8_5, sum(γ[1:2^nplants8_5, t]) == 1)
end

for plant in 1:nplants8_5
    @constraint(m8_5, sum(f[plant, 1:N_t8_5]) == N_t8_5 - 1)
end
for t in 1:N_t8_5
    for j in 0:(2^nplants8_5 - 1)
        j_bin = Base.bin(Unsigned(j), nplants8_5, false)
        j_array = [parse(Int, j_bin[end + 1 - plant]) for plant in 1:nplants8_5]
        @constraint(m8_5, γ[j + 1, t] >= sum((-1)^(1 + j_array[plant]) * f[plant, t] for plant in 1:nplants8_5) + 1 - count_ones(j))
    end
end

@objective(m8_5, Max, sum(Pc));

In [41]:
JuMP.build(m8_5)
Xpress.setlogfile(m8_5.internalModel.inner,"D:/m8_5.log")

In [ ]:
@time solve(m8_5)
@show schedule_matrix8_5 = getvalue(f)
@show getvalue(Pc);

In [ ]:
dados8_5, heatmap8_5, graph8_5 = dados_e_plot(G8_5, schedule_matrix8_5, d8_5, legend = true, inverse = true)
dados8_5